# What's Covered?

- Some basic examples of using Dash in a notebook
    - How to interact with a single function
    - How to add a dependent function
    - How to display Graphs with additional interactivity (download etc)
    - How to dynamically update Graphs based on inputs
    - How to work with Data objects in Dash (serverside)
    - How to work with Dat objects in Dash (serverside)
    


# Using Dash in Jupyterlab

To make Dash work well with a fairly normal analysis workflow, in addition to `dash` we need two additional packages:
- `jupyter-dash` -- Allows re-running app building/serving without restarting the jupyter kernel
- `dash-extensions` -- Provides a way to keep outputs serverside (ServersideOutput) which makes datatypes more flexible
- `jupyter-server-proxy` -- Allows viewing the Dash app while connected remotely

Because both `jupyter-dash` and `dash-extensions` requires using their own subclass of the `Dash` class, we will combine them and use combined class instead.

These packages have very loose dependencies (i.e. they don't specify what they need) but apparently very strict requirements (i.e. they fail if they don't have exactly what they need)

Below is a list of the main packages that are relevant and versions that definitely work together:

# Generally Useful/Required Imports
I.e. You'll usually want to use these next few cells when implementing dash interaction in a notebook page

In [1]:
from dat_analysis.dash.util import MyDash, Components as C, make_app, make_layout_section, get_unused_port
import dash
import dash_bootstrap_components as dbc

def run_app(app, port:int = None, run=True, mode: str = 'inline'):
    """
        For ease of use running app with correct settings (and easy to toggle run behavior)
        Note: This function MUST stay in the notebook that calls it (something to do with the jupyter proxy stuff)
    """
    import logging
    if run:
        if (count := getattr(app, '_run_count', 0)) > 0:
            logging.warning(f'Note: app has been run {count} times previously, this can affect callback behavior')
        app._run_count = count + 1
        port = port if port else get_unused_port()
        app.run_server(mode=mode, port=port)
    else:
        print(f'To run the app at this point, set `run=True`')

In [3]:
raise Exception(f'\n\n>>>>> Note: `JupyterDash.infer_jupyter_proxy_config()` has to be run with Shift+Enter (not the "Run All" button) <<<<<\n>>>>> This exception is here to enforce that <<<<<')

Exception: 

>>>>> Note: `JupyterDash.infer_jupyter_proxy_config()` has to be run with Shift+Enter (not the "Run All" button) <<<<<
>>>>> This exception is here to enforce that <<<<<

In [2]:
# This cell must be run as a single cell (Shift/Ctrl+Enter) any time before `run_app(app)` is called. 
from jupyter_dash import JupyterDash
# This allows the dash app to work when connected remotely
# Note: For some reason, running this line is incomptible with using the `lab_black` extension. (and lab_black cannot be loaded later)
%unload_ext lab_black
# JupyterDash.infer_jupyter_proxy_config()

# Note: If there are issues with this cell, try restarting the kernel and waiting a few seconds before executing this cell

The lab_black extension is not loaded.


# Extremely basic tests to check the dash app works at all

In [3]:
import pandas as pd
import numpy as np
from dash import html
import plotly.graph_objects as go

from dat_analysis.plotting.plotly.util import figures_to_subplots

In [4]:
# Test that the simplest callback works
app = make_app()

# Make a simple layout
output = C.Div(id='df-out')
in_a = C.Input('in-a', value=0)
app.layout = html.Div(children=[in_a, output])

# Note: If the function is only going to be used for a single callback it can be decorated like this
# Note: however, it makes the `test_func` unusuable outside of the app (test_func will == None)
@app.callback(output.as_output(), in_a.as_input())
def test_output(a):
    return a

# Run the app
run_app(app, run=False)

To run the app at this point, set `run=True`


# Example of working with more general functions

## Interacting with a single function
### Make/test a function as if it was being used in a normal analysis page

In [5]:
# Now test a basic function that doesn't return a json serializable output (e.g. that would usually be required by dash)
def make_df_test(a, b):
    """E.g. Some part of normal data processing"""
    df = pd.DataFrame(data=[a, b])
    return df
    
# Example Usage: Use the function for normal processing or for testing it works as expected
test_df = make_df_test(3,4)
test_df

,0
0,3
1,4


### How to interact with that function through a Dash App

Note: The `serverside` output here is the important part that allows for passing a non json-serializable object from a callback

In [6]:
# Make outputs (place to store output of function and to show the output)
df_store = C.Store()
df_markdown = C.Markdown(heading='Test DF Output')

# Make inputs (to fill values the function takes)
in_a = C.Input('Input A')  # Note: This is also the ID of the input
in_b = C.Dropdown('Input B', options=[1,2,3,4,5], value=3)

# Put those components together into a layout
layout = make_layout_section(stores=[df_store], inputs=[in_a, in_b], outputs=[df_markdown])

# Wrap the function (necessary if the inputs need tidying before passing to real function)
def _make_df_test(a, b):
    # Make sure inputs are floats (dash passes strings for some types of input)
    a, b = [-1 if v is None else float(v) for v in [a,b]]
    return make_df_test(a, b)

# Define a function that displays the output of make_df_test
def show_df(df: pd.DataFrame):
    """Convert df to something to show in Dash App"""
    return df.to_markdown()

# Make app and attach callbacks (Note: cannot re-run callbacks with existing app)
app = make_app()
app.layout = layout  # Assign app layout (in this case just this section, but could include more)
app.callback(df_store.as_output(serverside=True), in_a.as_input(), in_b.as_input())(_make_df_test)
app.callback(df_markdown.as_output(), df_store.as_input())(show_df)

# Run the app
run_app(app, run=False)

To run the app at this point, set `run=True`


## Adding another function that depends on first
### Make another function

In [7]:
# Example of using another function that takes the output of the first plus a new input
def do_something_with_df(df: pd.DataFrame, multiplier):
    return df.apply(lambda x: x*multiplier)

# Example Usage: Use the function for normal processing or for testing it works as expected
do_something_with_df(test_df, 4)

,0
0,12
1,16


### Continue to add to same app created above

In [8]:
# Make outputs (place to store output of function and to show the output)
multiplied_df_store = C.Store()
multiplied_df_markdown = C.Markdown(heading='Multiplied DF Output')

# Make inputs (to fill values the function takes)
multiplier_slider = C.Slider('Multiplier', min=1, max=10, step=1, value=3)

# Put those components together into a layout
mult_layout = make_layout_section(stores=[multiplied_df_store], inputs=[multiplier_slider], outputs=[multiplied_df_markdown])

# Wrap the function (necessary if the inputs need tidying before passing to real function)
def _do_something_with_df(df: pd.DataFrame, multiplier):
    multiplier = 0 if multiplier is None else multiplier
    return do_something_with_df(df, multiplier)

# Re-assign layout and attach callbacks (Note: cannot re-run callbacks with existing app)
# Note: Not making a new app here so as not to have to re-run the callbacks in cells above
app.layout = dbc.Container([layout, mult_layout])  # Combining layouts
app.callback(multiplied_df_store.as_output(serverside=True), df_store.as_input(), multiplier_slider.as_input())(_do_something_with_df)
app.callback(multiplied_df_markdown.as_output(), multiplied_df_store.as_input())(show_df)  # Note: Re-using earlier function

# Run the app
run_app(app, run=True)

Dash is running on http://127.0.0.1:57829/



# Example with Graphs

## Displaying Static Figures
I.e. Figures that do not get updated by other functions

### Make some test figures to show

In [9]:
x = np.linspace(0, 2)
y = np.linspace(0, 1)
z = np.sin(np.exp(x))*np.cos(y**2)[:, None]

fig = go.Figure().add_trace(go.Heatmap(x=x, y=y, z=z)).update_layout(title='Testing a 2D Figure')
fig2 = go.Figure().add_trace(go.Scatter(x=x, y=np.sin(4*y))).update_layout(title='Testing a 1D Figure')

# Change to True to plot the regular figures
if False:
    fig.show()
    fig2.show()

### Add to app
Once added to the app, you will get access to options to download the figures or toggle between waterfall mode etc.

In [10]:
# Make outputs that hold figures
graph1 = C.Graph(figure=fig)  
graph2 = C.Graph(figure=fig2)

# Make a new app
app = make_app()

# Add figures to layout
app.layout = C.Div(children=[graph1.layout(), graph2.layout()])

# Add callbacks
C.Graph.run_callbacks(app)  # Note: this will work for all C.Graph components (they share the same callback)

# Run app
run_app(app, run=False)

To run the app at this point, set `run=True`


## Displaying Dynamic Figures
I.e. Figures that can be updated by other functions

### Make a test function that makes figures

In [11]:
def make_test_fig(a: float, b: float):
    x = np.linspace(0, a)
    y = np.linspace(0, b)
    z = np.sin(np.exp(x))*np.cos(y**2)[:, None]
    fig = go.Figure().add_trace(go.Heatmap(x=x, y=y, z=z)).update_layout(title='Testing a 2D Figure')
    return fig

# Change to True to plot the regular figures
if False:
    make_test_fig(2, 7).show()

### Add to app

In [12]:
# Make outputs (place to store output of function and to show the output)
graph1 = C.Graph()

# Make inputs (to fill values the function takes)
slider_a = C.Slider('Input A', min=0.1, max=5, value=2)
slider_b = C.Slider('Input B', min=1, max=15, value=2)

# Put those components together into a layout
layout = make_layout_section(stores=[], inputs=[slider_a, slider_b], outputs=[graph1])

# Wrap the function (necessary if the inputs need tidying before passing to real function)
# Note: Not necessary with sliders (they already provide floats)

# Make app, assign layout, and attach callbacks (Note: cannot re-run callbacks with existing app)
app = make_app()
app.layout = layout
C.Graph.run_callbacks(app)
app.callback(graph1.as_output(), slider_a.as_input(), slider_b.as_input())(make_test_fig)

# Run the app
run_app(app, run=True, mode='inline')

Dash is running on http://127.0.0.1:57860/



# Example with Data objects 
i.e. `Data(x=np.array, y=np.array, data=np.array)`

### Standard Set-up of get_dat for Examples
The standard set-up of `get_dat` for these examples, you would not normally do this in every individual .ipynb file

In [13]:
import dat_analysis

def get_dat(datnum, raw=False, overwrite=False):
    """
    Define a simple get_dat function that knows where to load dats from and save them to

    Note: In a full setup where a path to `measurement-data` and a general save location have already been set in the dat_analysis config.toml, you can use:
        return dat_analysis.get_dat(datnum, host_name, user_name, experiment_name, raw=raw, overwrite=overwrite)
    """
    hdf_name = f"dat{datnum}{'_RAW' if raw else ''}.h5"
    return dat_analysis.get_dat_from_exp_filepath(f'experiment_dats/{hdf_name}', override_save_path=f'analyzed_dats/{hdf_name}', overwrite=overwrite)

In [14]:
from dat_analysis import Data

# For type checking
from dat_analysis.dat.dat_hdf import DatHDF

In [15]:
# Load some dats to test with
dats = [get_dat(6420), get_dat(6507)]

# Avoid reloading dats where possible (although not really necessary)
dat_dict = {dat.datnum: dat for dat in dats}

## Passing a single Data instance

In [16]:
#### To display this function  ####
def get_cs_data_from_dat(dat: DatHDF):
    x = dat.Data.x
    y = dat.Data.y
    data = dat.Data.get_data('cscurrent_2d')
    return Data(x=x, y=y, data=data)
####################################

# Make outputs (place to store output of function and to show the output)
store = C.Store()  
graph = C.Graph()

# Make inputs (to fill values the function takes)
in_a = C.Dropdown('Select Datnum', options=[dat.datnum for dat in dats], value=dats[0].datnum)

# Put those components together into a layout
layout = make_layout_section(stores=[store], inputs=[in_a], outputs=[graph])

# Wrap the function (necessary if the inputs need tidying before passing to real function)
def _get_cs_data_from_dat(datnum: int):
    if datnum in dat_dict:
        dat = dat_dict[datnum]
        return get_cs_data_from_dat(dat)
    else:
        logging.warning(f'{datnum} not in dat_dict')
        return dash.no_update

# Define a function that displays the output of make_df_test
def show_output(data: Data):
    """Convert df to something to show in Dash App"""
    if data is None:
        return dash.no_update
    if data.data.ndim == 1:
        fig = go.Figure().add_trace(go.Scatter(x=data.x, y=data.data))
    elif data.data.ndim == 2:
        fig = go.Figure().add_trace(go.Heatmap(x=data.x, y=data.y, z=data.data))
    else:
        logging.warning(f'{data.data.ndim} not 1 or 2')
    return fig

# Make app and attach callbacks (Note: cannot re-run callbacks with existing app)
app = make_app()
app.layout = layout  
app.callback(store.as_output(serverside=True), in_a.as_input())(_get_cs_data_from_dat)
app.callback(graph.as_output(), store.as_input())(show_output)
C.Graph.run_callbacks(app)

# Run the app
run_app(app, run=True)

Dash is running on http://127.0.0.1:57183/



## Passing a list of Data instances

In [17]:
from dat_analysis.useful_functions import ensure_list

#### To display this function  ####
def get_cs_data_from_dats(dats: list[DatHDF]) -> list[Data]:
    datas = []
    for dat in dats:
        x = dat.Data.x
        y = dat.Data.y
        data = dat.Data.get_data('cscurrent_2d')
        datas.append(Data(x=x, y=y, data=data))
    return datas
####################################

# Make outputs (place to store output of function and to show the output)
store = C.Store()  
out_div = C.Div()

# Make inputs (to fill values the function takes)
in_a = C.Dropdown('Select Datnum', options=[dat.datnum for dat in dats], value=[dat.datnum for dat in dats[:3]], multi=True)

# Put those components together into a layout
layout = make_layout_section(stores=[store], inputs=[in_a], outputs=[out_div])

# Wrap the function (necessary if the inputs need tidying before passing to real function)
def _get_cs_data_from_dats(datnums: list[int]):
    datnums = ensure_list(datnums)
    dats = []
    for datnum in datnums:
        if datnum in dat_dict:
            dat = dat_dict[datnum]
            dats.append(dat)
        else:
            logging.warning(f'{datnum} not in dat_dict')
    return get_cs_data_from_dats(dats)

# Define a function that displays the output of make_df_test
def show_output(datas: list[Data]):
    """Convert df to something to show in Dash App"""
    if datas is None or len(datas) == 0:
        return dash.html.Div('Select a dat to show output')
    figs = []
    for data in datas:
        if data.data.ndim == 1:
            fig = go.Figure().add_trace(go.Scatter(x=data.x, y=data.data))
        elif data.data.ndim == 2:
            fig = go.Figure().add_trace(go.Heatmap(x=data.x, y=data.y, z=data.data))
        else:
            logging.warning(f'{data.data.ndim} not 1 or 2')
        figs.append(fig)
    return dash.html.Div(children=[C.Graph(figure=fig) for fig in figs])

# Make app and attach callbacks (Note: cannot re-run callbacks with existing app)
app = make_app()
app.layout = layout  
app.callback(store.as_output(serverside=True), in_a.as_input())(_get_cs_data_from_dats)
app.callback(out_div.as_output(), store.as_input())(show_output)
C.Graph.run_callbacks(app)

# Run the app
run_app(app, run=True)

Dash is running on http://127.0.0.1:57187/



# Example working with DatHDF objects
Even Dat objects can be passed through Store components as long as they are updated with Serverside callbacks

## Passing a Single DatHDF object

In [18]:
def select_dat(datnum) -> DatHDF:
    if datnum in dat_dict:
        return dat_dict[datnum]
    return None

def get_cs_data_from_dat(dat: DatHDF):
    if dat is not None:
        x = dat.Data.x
        y = dat.Data.y
        data = dat.Data.get_data('cscurrent_2d')
        return Data(x=x, y=y, data=data)
    return None

#############################


# Make outputs (place to store output of function and to show the output)
stores = [
    dat_store := C.Store(),
    data_store := C.Store(),
]
outputs = [
    graph := C.Graph(),
]
# Make inputs (to fill values the function takes)
inputs = [
    in_a := C.Dropdown('Select Datnum', options=[dat.datnum for dat in dats], value=dats[0].datnum),
]

# Put those components together into a layout
layout = make_layout_section(stores=stores, inputs=inputs, outputs=outputs)

# Wrap the function (necessary if the inputs need tidying before passing to real function)
# Not necessary

# Define a function that displays the output of make_df_test
def show_output(data: Data, dat: DatHDF):
    """Convert df to something to show in Dash App"""
    if data is None:
        return go.Figure()
    if data.data.ndim == 1:
        fig = go.Figure().add_trace(go.Scatter(x=data.x, y=data.data))
    elif data.data.ndim == 2:
        fig = go.Figure().add_trace(go.Heatmap(x=data.x, y=data.y, z=data.data))
    else:
        logging.warning(f'{data.data.ndim} not 1 or 2')
        fig = go.Figure()
    if dat is not None:
        fig.update_layout(title=f'Dat{dat.datnum}: Example Title <br>Time Completed={dat.Logs.time_completed}', xaxis_title=dat.Logs.x_label, yaxis_title=dat.Logs.y_label)
    return fig

# Make app and attach callbacks (Note: cannot re-run callbacks with existing app)
app = make_app()
app.layout = layout  
app.callback(dat_store.as_output(serverside=True), in_a.as_input())(select_dat)
app.callback(data_store.as_output(serverside=True), dat_store.as_input())(get_cs_data_from_dat)
app.callback(graph.as_output(), data_store.as_input(), dat_store.as_input())(show_output)
C.Graph.run_callbacks(app)

# Run the app
run_app(app, run=True)

Dash is running on http://127.0.0.1:57191/



## Pass a List of DatHDF objects

In [19]:
def select_dats(datnums) -> DatHDF:
    dats = []
    if datnums is not None:
        for datnum in datnums:
            if datnum in dat_dict:
                dats.append(dat_dict[datnum])
    return dats

def get_cs_data_from_dats(dats: list[DatHDF]) -> list[Data]:
    datas = []
    for dat in dats:
        x = dat.Data.x
        y = dat.Data.y
        data = dat.Data.get_data('cscurrent_2d')
        datas.append(Data(x=x, y=y, data=data))
    return datas
####################################

# Make outputs (place to store output of function and to show the output)
stores = [
    dats_store := C.Store(),
    data_store := C.Store(),
]
outputs = [
    out_div := C.Div(),
]
# Make inputs (to fill values the function takes)
inputs = [
    in_a := C.Dropdown('Select Datnum', options=[dat.datnum for dat in dats], value=[dat.datnum for dat in dats[:3]], multi=True)
]

# Put those components together into a layout
layout = make_layout_section(stores=stores, inputs=inputs, outputs=outputs)

# Wrap the function (necessary if the inputs need tidying before passing to real function)
# Not necessary

# Define a function that displays the output of make_df_test
def show_output(datas: list[Data], dats: list[DatHDF]):
    """Convert df to something to show in Dash App"""
    if datas is None or len(datas) == 0:
        return dash.html.Div('Select a dat to show output')
    figs = []
    for data, dat in zip(datas, dats):
        if data.data.ndim == 1:
            fig = go.Figure().add_trace(go.Scatter(x=data.x, y=data.data))
        elif data.data.ndim == 2:
            fig = go.Figure().add_trace(go.Heatmap(x=data.x, y=data.y, z=data.data))
        else:
            logging.warning(f'{data.data.ndim} not 1 or 2')
            fig = default_fig()
        fig.update_layout(title=f'Dat{dat.datnum}: Example Title <br>Time Completed={dat.Logs.time_completed}', xaxis_title=dat.Logs.x_label, yaxis_title=dat.Logs.y_label)
        figs.append(fig)
    return dash.html.Div(children=[C.Graph(figure=fig) for fig in figs])

# Make app and attach callbacks (Note: cannot re-run callbacks with existing app)
app = make_app()
app.layout = layout  
app.callback(dats_store.as_output(serverside=True), in_a.as_input())(select_dats)
app.callback(data_store.as_output(serverside=True), dats_store.as_input())(get_cs_data_from_dats)
app.callback(out_div.as_output(), data_store.as_input(), dats_store.as_input())(show_output)
C.Graph.run_callbacks(app)

# Run the app
run_app(app, run=True)

Dash is running on http://127.0.0.1:57207/

